In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.assignment_models as Models
import core
import keras as keras
import tensorflow as tf
import os


PLOTS_DIR = f"plots/toponium_transformer/"
MODEL_DIR = f"models/toponium_transformer/"
CONFIG_PATH = "../config/workspace_config.yaml"

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)

plt.rcParams.update({"font.size": 14})

data_config = DataProcessor.load_data(
    load_config["toponium"], "reco", max_events=1_000_000
)
X_train, y_train, X_val, y_val = DataProcessor.split_data(test_size=0.1)
del DataProcessor

2025-11-14 09:42:37.785382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763109757.809279 3962043 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763109757.817133 3962043 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763109757.835727 3962043 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763109757.835746 3962043 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763109757.835748 3962043 computation_placer.cc:177] computation placer alr

In [2]:
data_config.get_feature_index("non_training","truth_ttbar_mass")

2

In [3]:
reload(Models)
reload(core)
TransformerMatcher = Models.FeatureConcatTransformer(data_config, name="ToponiumTransformer")

#TransformerMatcher.load_model(MODEL_DIR + "model.keras")
TransformerMatcher.build_model(
    hidden_dim=64,
    num_layers=6,
    num_heads=8,
    dropout_rate=0.1
)

TransformerMatcher.adapt_normalization_layers(X_train)

TransformerMatcher.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(lambda_excl=0.0),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy()],
    },
)

FeatureConcatTransformer is designed for classification tasks; regression targets will be ignored.


I0000 00:00:1763109803.416235 3962043 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Building model without regression output.


In [4]:
TransformerMatcher.train_model(
    epochs=100,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
)

Epoch 1/100


I0000 00:00:1763109821.480173 3962496 service.cc:152] XLA service 0x7fd0d800f380 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763109821.480203 3962496 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-11-14 09:43:42.098128: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-14 09:43:42.778461: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
I0000 00:00:1763109824.537001 3962496 cuda_dnn.cc:529] Loaded cuDNN version 91500


  7/704 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - assignment_accuracy: 0.1324 - loss: 0.0015  

I0000 00:00:1763109838.457382 3962496 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


700/704 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - assignment_accuracy: 0.0263 - loss: 0.0011

2025-11-14 09:44:10.357738: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-11-14 09:44:23.627328: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_multiply_reduce_select_fusion', 120 bytes spill stores, 96 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_1', 104 bytes spill stores, 88 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion', 112 bytes spill stores, 92 bytes spill loads



704/704 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - assignment_accuracy: 0.0262 - loss: 0.0011

2025-11-14 09:44:25.125458: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-11-14 09:44:27.192624: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


704/704 ━━━━━━━━━━━━━━━━━━━━ 61s 43ms/step - assignment_accuracy: 0.0262 - loss: 0.0011 - val_assignment_accuracy: 0.0129 - val_loss: 9.3296e-04
Epoch 2/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - assignment_accuracy: 0.0310 - loss: 9.7560e-04 - val_assignment_accuracy: 0.0621 - val_loss: 9.0849e-04
Epoch 3/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - assignment_accuracy: 0.0848 - loss: 9.3766e-04 - val_assignment_accuracy: 0.1647 - val_loss: 8.8099e-04
Epoch 4/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - assignment_accuracy: 0.1402 - loss: 9.0974e-04 - val_assignment_accuracy: 0.1575 - val_loss: 8.6429e-04
Epoch 5/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - assignment_accuracy: 0.2235 - loss: 8.8480e-04 - val_assignment_accuracy: 0.3595 - val_loss: 8.3041e-04
Epoch 6/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - assignment_accuracy: 0.3370 - loss: 8.5034e-04 - val_assignment_accuracy: 0.4146 - val_loss: 7.9118e-04
Epoch 7/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/

In [5]:
TransformerMatcher.save_model(MODEL_DIR + "model.keras")

Model saved to models/toponium_transformer/model.keras
